In [1]:
# Aminov Rezo
import numpy as np
import wave
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
import random

import struct

from keras.models import *
from keras.layers import *
from keras.callbacks import *

import librosa
import soundfile as sf
from keras.models import load_model

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

/home/aminov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_DIR = 'data_v_7_stc'
meta_file = "{}/meta/meta.txt".format(DATA_DIR)
df = pd.read_csv(meta_file, sep='\t',header=None)
labels_name = df[4].unique()

In [3]:
# кодирование лейблов
onehot_dict = {}
for ii, lab in enumerate(labels_name):
    y_ = np.zeros(len(labels_name))
    y_[ii] = 1
    onehot_dict.update({lab:ii})
    
# обратный хэш
hot_to_one = {}
for k,v in onehot_dict.items():
    hot_to_one.update({v:k})

In [4]:
# экстрактор фич: Мел-кепстральные коэффициенты (MFCC). https://habr.com/post/140828/
def extract_feature(file_name):
    X, sample_rate = sf.read(file_name, dtype='float32')
    if X.ndim > 1:
        X = X[:,0]
    X = X.T
    # преобразование Фурье
    stft = np.abs(librosa.stft(X))
    # MFCC
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=128).T,axis=0)
    # chroma
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # мэл спектр
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # спектр-ный контраст
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,mel

In [5]:
files_test = glob(DATA_DIR+'/test/*.wav')

In [6]:
model = load_model('weights/model.hdf5')
model.load_weights('weights/model_weights.hdf5')

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 254, 128)          512       
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 250, 128)          82048     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 83, 128)           0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 77, 128)           114816    
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 69, 128)           147584    
_________________________________________________________________
global_average_pooling1d_6 ( (None, 128)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

In [8]:
CNT=0 # кол-во всех не 'unknown', подмножество 'A'
GOOD=0 # кол-во правильно опред-ых файлов в подмножестве 'A'
BAD=0 # кол-во не правильно опред-ых файлов в подмножестве 'A'

filew = open("result.txt","a") 
features_test = np.empty((0,256))
for file in files_test:    
    try:
        mfccs, mel= extract_feature(file)
    except Exception as e:
        print("[Error] extract feature error. %s" % (e))
        continue
    ext_features_test = np.hstack([mfccs, mel])
#     features_test = np.vstack([features_test,ext_features_test])
    pred = model.predict(np.expand_dims([ext_features_test],axis=2))
    score = pred.max()
    class_ = hot_to_one[np.argmax(pred)]
    filename = file.split('/')[2]
    
    filew.write(filename+'\t'+str(score)+'\t'+class_+'\n')
    print(filename+' '+str(score)+' '+class_)
    
    # если файл не 'unknown', делаю подсчет совпадений лейбла и наз. файла
    # примерный подсчет, т.к. неизвестно к какому классу относятся файлы
    # с наз. 'unknown'
    if 'unknown' not in filename:
        CNT+=1
        if class_ in filename:
            GOOD+=1
        else:
            BAD+=1
    
filew.close()

unknown_0070.wav 0.9994543 tool
bags_t_0019.wav 0.9999999 bags
door_0085.wav 1.0 door
background_0060.wav 0.99998724 background
background_0036.wav 0.83218956 background
speech_0086.wav 0.9984975 speech
bags_0038.wav 1.0 bags
speech_0074.wav 1.0 speech
unknown_0034.wav 1.0 bags
speech_t_0031.wav 1.0 speech
knocking_door_0066.wav 1.0 knocking_door
knocking_door_t_0003.wav 1.0 knocking_door
knocking_door_0060.wav 1.0 knocking_door
bags_0086.wav 1.0 bags
unknown_0033.wav 0.39525914 bags
knocking_door_t_0007.wav 0.9975732 door
background_t_0011.wav 0.5966699 background
keyboard_t_0001.wav 0.46165976 door
bags_t_0023.wav 0.99999976 bags
unknown_0071.wav 0.99999917 tool
unknown_0089.wav 0.99999785 keyboard
keyboard_0026.wav 0.9999851 keyboard
speech_0008.wav 0.9999999 speech
speech_0006.wav 0.9996927 speech
unknown_0115.wav 0.9999999 tool
background_0022.wav 0.97707444 background
knocking_door_0018.wav 0.94909203 knocking_door
speech_0071.wav 0.99955624 speech
tool_0004.wav 1.0 tool
speech_0

knocking_door_t_0015.wav 0.99940264 knocking_door
keyboard_t_0011.wav 0.9004449 keyboard
unknown_0041.wav 0.98227596 knocking_door
keyboard_t_0015.wav 0.9913282 keyboard
ring_0002.wav 0.99999917 ring
unknown_0137.wav 0.99995494 tool
background_0063.wav 0.999485 background
keyboard_0046.wav 0.99995136 keyboard
unknown_0102.wav 0.7922409 background
bags_t_0012.wav 1.0 bags
unknown_0075.wav 0.8951627 keyboard
background_t_0001.wav 0.6920507 background
bags_0082.wav 1.0 bags
background_t_0013.wav 0.9977602 background
keyboard_0008.wav 1.0 keyboard
unknown_0087.wav 0.69307494 background
knocking_door_t_0017.wav 0.9999987 door
ring_0048.wav 0.99923825 ring
door_t_0017.wav 0.9999995 door
speech_0031.wav 0.99999774 speech
speech_0083.wav 1.0 speech
ring_0022.wav 1.0 ring
door_0061.wav 1.0 background
knocking_door_0057.wav 1.0 knocking_door
unknown_0004.wav 0.59447265 knocking_door
unknown_0118.wav 0.7227891 speech
ring_0003.wav 0.9999802 ring
speech_0087.wav 1.0 speech
speech_0085.wav 1.0 spee

unknown_0078.wav 0.8288034 background
knocking_door_0026.wav 1.0 knocking_door
speech_0095.wav 0.99999905 speech
unknown_0114.wav 0.8165755 tool
unknown_0073.wav 0.6939071 keyboard
unknown_0106.wav 0.6486429 tool
unknown_0120.wav 0.6111065 knocking_door
speech_t_0037.wav 1.0 speech
bags_0017.wav 1.0 bags
unknown_0060.wav 0.9995567 tool
unknown_0017.wav 0.99983394 tool
door_t_0020.wav 0.99934334 door
tool_0007.wav 0.99997056 tool
unknown_0062.wav 1.0 tool
ring_0020.wav 1.0 ring
background_0012.wav 0.9916291 background
ring_t_0004.wav 0.999998 ring
unknown_0111.wav 0.9825936 tool
unknown_0100.wav 0.80486184 keyboard
unknown_0064.wav 1.0 tool
unknown_0023.wav 0.68917924 door
door_t_0008.wav 1.0 door
background_0051.wav 0.99523866 background
keyboard_0012.wav 0.99999964 keyboard
background_t_0005.wav 0.99857557 background
knocking_door_0014.wav 1.0 knocking_door
background_0046.wav 0.9988525 background
speech_0037.wav 1.0 speech
keyboard_t_0022.wav 0.8747212 keyboard
speech_0070.wav 0.9999

In [9]:
CNT, GOOD, BAD

(473, 437, 36)

In [10]:
GOOD/CNT

0.9238900634249472